In [1]:
import pickle
import numpy as np
from flask import Flask, jsonify, request

### 4. Flask : usando JSON

Recordemos estos argumentos para el open :

    'math_model.pkl' > El archivo que contiene el modelo serializado.

    'rb' > Es la forma de lectura en binario.

Para deserializar un pickle, debemos mantener abierto el archivo que contiene la data en forma de lectura binaria.

In [2]:
# vamos a cargar uno de los modelos en memoria a partir de un archivo pickle
# el archivo se mantiene abierto solo en el ambito del with, despues del bloque se cierra automaticamente.
with open('math_model.pkl', 'rb') as f_math:
    
    modelo_matematicas = pickle.load(f_math)

modelo_matematicas

ElasticNetCV(alphas=array([1.000e-03, 1.020e-01, 2.030e-01, 3.040e-01, 4.050e-01, 5.060e-01,
       6.070e-01, 7.080e-01, 8.090e-01, 9.100e-01, 1.011e+00, 1.112e+00,
       1.213e+00, 1.314e+00, 1.415e+00, 1.516e+00, 1.617e+00, 1.718e+00,
       1.819e+00, 1.920e+00, 2.021e+00, 2.122e+00, 2.223e+00, 2.324e+00,
       2.425e+00, 2.526e+00, 2.627e+00, 2.728e+00, 2.829e+00, 2.930e+00,
       3.031e+00, 3.132e+00, 3.2...
       8.485e+00, 8.586e+00, 8.687e+00, 8.788e+00, 8.889e+00, 8.990e+00,
       9.091e+00, 9.192e+00, 9.293e+00, 9.394e+00, 9.495e+00, 9.596e+00,
       9.697e+00, 9.798e+00, 9.899e+00, 1.000e+01]),
             copy_X=True, cv=3, eps=0.001, fit_intercept=True, l1_ratio=0.5,
             max_iter=1000, n_alphas=100, n_jobs=-1, normalize=False,
             positive=False, precompute='auto', random_state=None,
             selection='cyclic', tol=0.0001, verbose=1)

Para poner el modelo a disposición, debemos tener en cuenta como vamos a definir la comunicación

En nuestro caso vamos a responder a un **POST requests** cuyo contenido va a ser un json como este :

**{ "model" : "math",**
 
 **"dummies": \["1", "0", "1", "0", "0", "0", "0", "1", "1", "0", "0", "0", "1", "0", "1", "0", "1"\] }**
 
 En donde cada posicion de un elemento en el array representa una dummy ordenada.

Para poder trabajar con este json debemos parsear el contenido del post requests (o sea el contenido que nos va a solicitar un potencial cliente).

Vamos a utilizar la funcionalidad dada por la clase _requests_ de flask para tratar la petición.

La clase _requests_ tiene un metodo *get_json* que permite parsear el contenido de un request especificado como json.

In [5]:
# http://flask.pocoo.org/docs/1.0/quickstart/

app = Flask('Predictor de examenes')

# tell Flask what URL should trigger our function
@app.route('/',methods=['POST'])
def predict():
    
    
    # https://tedboy.github.io/flask/generated/generated/flask.Request.get_json.html
    
    # obtengo los datos del request post.
    # notar que en este contexto request contiene la informacion 
    # que viene de la peticion externa (el metodo get_json lo transforma en un diccionario)
    data = request.get_json(force=True)
    
    
    # transformamos el dato del json (un array de string) en un array de enteros de numpy
    # para que lo entienda el modelo, notar la forma del array y la transformacion de los tipos de datos
    X_para_prediccion = np.array(data['dummies']).reshape(1, -1).astype('int')
    
    # implementamos una logica en donde elegimos un modelo (podrian ser los otros modelos, lect/escritura)
    if data['model'] == 'math':
        
        # asignamos el array de numpy que nos devuelve una prediccion
        output = modelo_matematicas.predict(X_para_prediccion)
                
        # esta prediccion es un array de un solo elemento
        prediccion = output[0]
        
        # Le damos forma de un diccionario para poder hacer el traspaso a json trivialmente
        output = {'prediccion': prediccion}
        
        # en esta linea, transformamos el diccionario en json con jsonify (funcionalidad de flask)
        # y respondemos el request con un json mediante este return
        # este json es incorporado en el cuerpo de la respuesta
        return jsonify(output)
    
    
@app.route('/hola',methods=['GET'])
def saludar():
    return "hola paula"

### En este punto vamos a poner a escuchar al servidor en 0.0.0.0:5000

Entre las cosas que vamos a ver por la salida estandar, va a ser la interaccion de nuestro servidor, con las peticiones externas. 

cuando sea ejecutada la proxima celda, la notebook se va a encontrar en estado de ejecución respondiendo peticiones, en este estado se le puede realizar un requests

In [ ]:
# el puerto 5000 es por default
app.run(host='0.0.0.0', port=5000 )

 * Serving Flask app "Predictor de examenes" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [08/Jul/2019 13:09:40] "GET /hola HTTP/1.1" 200 -
127.0.0.1 - - [08/Jul/2019 13:09:50] "GET /hola HTTP/1.1" 200 -
127.0.0.1 - - [08/Jul/2019 13:09:50] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [08/Jul/2019 13:10:27] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [08/Jul/2019 13:10:54] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [08/Jul/2019 13:11:37] "POST / HTTP/1.1" 200 -
